## First we initialize our globals

Inititalizes the `db.cow_bff` table with sample cow eating time data.

Run this notebook from the web UI or by executing the `Databricks: Run File as Workflow on Databricks` command.

In [0]:
import random

start_epoch = 1684540800
end_epoch = 1685620800
day = 86400

cows = ['Butterscotch','Hershey','Mocha','Nutella','Brandy','Peaches','Marshmallow','Popcorn','Muffin','Daisy','Buttercup','Dottie','Magic','Nellie',]

meal_1_start = 32400

#we will only focus on one meal
meal_2_start = 64800

meal_duration_start = -600
meal_duration_end = 2400



In [0]:
%sql
use catalog farm;

In [0]:
%sql
create schema if not exists db;

In [0]:
%sql
drop table if exists db.cows_bff;

In [0]:
index = start_epoch
import pandas as pd
import datetime
import calendar
meal_data2 = None
data =[]
while True:
  if index > end_epoch:
    break;
  date = datetime.datetime.fromtimestamp(index)
  for cow in cows:
    r1 = random.randint(meal_duration_start, meal_duration_end)
    r2 = random.randint(meal_duration_start, meal_duration_end)
    begin = r1
    end = r2
    if r1 > r2:
      begin = r2
      end = r1
    meal_start_seconds = round(meal_1_start+begin)
    meal_end_seconds = round(meal_1_start+end)
    meal_data1 = {
      'cow_name': cow,
      'meal_start': meal_start_seconds, 
      'meal_end': meal_end_seconds,
      'meal_start_time':str(datetime.timedelta(seconds=meal_start_seconds)), 
      'meal_end_time':str(datetime.timedelta(seconds=meal_end_seconds)),
      'duration': meal_end_seconds-meal_start_seconds,
      'date': date,
      'day': calendar.day_name[date.weekday()]}
    data.append(meal_data1)
    # we cheat and add Mocha's best friend Cocoa
    if(cow=='Mocha'):
      cocoa_meal_start_seconds = round(meal_start_seconds - (random.randint(0,500)/10000.0))
      cocoa_meal_end_seconds = round(meal_end_seconds- (random.randint(0,500)/10000.0))
      meal_data2 = {
        'cow_name': 'Cocoa',
        'meal_start': cocoa_meal_start_seconds, 
        'meal_end': cocoa_meal_end_seconds,
        'meal_start_time': str(datetime.timedelta(seconds=cocoa_meal_start_seconds)), 
        'meal_end_time': str(datetime.timedelta(seconds=cocoa_meal_end_seconds)),
        'duration': cocoa_meal_end_seconds-cocoa_meal_start_seconds,
        'date': date,
        'day': calendar.day_name[date.weekday()]}
      data.append(meal_data2)
    
  index = index + day
  df = pd.DataFrame(data)
  

In [0]:
# save data into table
spark_df = spark.createDataFrame(df)

## Write Frame out as Table
spark_df.write.mode("overwrite").saveAsTable("db.cows_bff")

In [0]:
display(spark_df)